In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/93855920110346950-senior-software-engineer-aiml-genai-google-cloud-ai")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Senior Software Engineer, AI/ML GenAI, Google Cloud AI — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results3,231  jobs matchedAccount Executive, Mid-Market Sales, Google Customer SolutionsMumbai, Maharashtra, India; Bengaluru, Karnataka, India; +2 more; +1 moreMicroLED Display Product EngineerFremont, CA, USAStaff Systems Development Engineering Manager, Google Public SectorReston, VA, USACPU Emulation EngineerAustin, TX, USA; Mountain View, CA, USA; +3 more; +2 moreSoftware Engineer III, Infrastructure, CoreBengaluru, Karnataka, IndiaSenior Software Engineer, AI/ML GenAI, Google Cloud AISunnyvale, CA, USA; Kirkland, WA, USASenior Programmatic SpecialistHyderabad, Telangana, India; Gurugram, Haryana, IndiaPro

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Senior Software Engineer, AI/ML GenAI, Google Cloud AI',
  'experience': 'Mid',
  'skills': ['Software development',
   'Data structures/algorithms',
   'GenAI techniques',
   'ML infrastructure'],
  'description': 'Write and test product or system development code, collaborate with peers and stakeholders, contribute to existing documentation, triage product or system issues, design and implement GenAI solutions.'},
 {'role': 'Account Executive, Mid-Market Sales, Google Customer Solutions',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'MicroLED Display Product Engineer',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Staff Systems Development Engineering Manager, Google Public Sector',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'CPU Emulation Engineer',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Software Engineer III, Infrastructure, Core',
  'experience': 

In [6]:
type(json_res)

list

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
links = collection.query(query_texts=["Experience in Python","Expertise in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [19]:
job = json_res[0]
job['skills']

['Software development',
 'Data structures/algorithms',
 'GenAI techniques',
 'ML infrastructure']

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}]]

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI/ML Solutions for Google Cloud AI - AtliQ

Dear Hiring Manager,

I came across the job description for a Senior Software Engineer, AI/ML GenAI, Google Cloud AI, and I am excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your requirements. With our expertise in software development, data structures/algorithms, GenAI techniques, and ML infrastructure, we are confident that we can provide top-notch solutions for your organization.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can collaborate with your peers and stakeholders to write and test product or system development code, contribute to existing documentation, triage product or system issues, and design and implement GenAI solutions.

I would like to highlight some of our relevant portfolio work that demonstrates our capabiliti